In [3]:
import os
import mokapot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
sys.path
import data_loader as dl


In [4]:
def filter_data(df, prob_column):
    #drop decoys
    df = df[df["decoy"]==False]
    #sort by qvalue
    df = df.sort_values(prob_column)
    #drop duplicate scans
    df = df.drop_duplicates(subset=["scan"], keep="first") #keep highest scoring
    #rename scan column
    df = df.rename(columns = {"scan": "ScanNr"})
    
    return df

In [2]:
#Reading in and formatting the original MetaMorpheus data
def get_orgininal_mm_data():
    mm_original_df = dl.clean_metamorph("2ng")
    mm_original_df = filter_data( mm_original_df, "QValue")
    
    #filter out just the ScanNr and QValue and/or PEP columns
    mm_original_df = mm_original_df.filter(items = ['ScanNr', 'peptide', 'QValue', 'PEP'])
    
    return mm_original_df

In [4]:
#Reading in and formatting the orginial MsFragger data
def set_probablility(row):
    new_prob = 1 - row.probability
    return new_prob

#pulling only scan numbers out
def extractScanNum(row):
    return(int(row[35:40]))

def get_original_msf_data():
    msf_original_df = dl.clean_msfragger("2ng")
    
    #Extracting scan number from file number
    msf_original_df['scan'] =msf_original_df['scan'].apply(extractScanNum) 
    
    msf_original_df = filter_data(msf_original_df, 'probability')
    
    #Changing the probabilities to the same scale all the other tools use 
    msf_original_df["Updated_probability"] =  msf_original_df.apply(set_probablility, 1)
    
    #filter out just the ScanNr and QValue and/or PEP columns
    msf_original_df = msf_original_df.filter(['ScanNr', 'peptide','probability'])
    
    return  msf_original_df

In [5]:
#Reading in and formatting the orginial MsgfPlus data
def get_original_msg_data():
    msg_original_df = dl.clean_msgfplus("2ng")
    msg_original_df = filter_data(msg_original_df, "QValue")
    
    #filter out just the ScanNr and QValue and/or PEP columns
    msg_original_df = msg_original_df.filter(["ScanNr", 'peptide', "QValue"])
    
    return msg_original_df

In [6]:
#Reading in and formatting the orginial MaxQuant data
def get_original_mq_data():
    mq_original_df =  dl.clean_maxquant("2ng")
    
#Formatting and dropping any rows that are missing the sequence
#     mq_original_df['Sequence'].replace(' ', np.nan, inplace = True)
#     mq_original_df.dropna(subset=['Sequence'], inplace=True)
    
    mq_original_df = filter_data(mq_original_df, 'PEP')
    
    #filter out just the ScanNr and QValue and/or PEP columns
    mq_original_df = mq_original_df.filter(["ScanNr", 'peptide', "PEP"])
    
    return  mq_original_df

In [7]:
##THIS DOESN'T WORK. WHY??
#Filter out the columns that we want to keep from the MokaPot results. 
def filter_MokaPot_data(df):
    df = df.filter(["ScanNr", 'mokapot score', "mokapot q-value", "mokapot PEP"])
    
    return df

In [ ]:
preMP_data = []
postMP_data = []

Reading in the results from running data through MokaPot 

In [8]:
msg_df = pd.read_csv('MokaPot_Output/msgfplus.csv')
msg_df = msg_df.filter(['ScanNr', 'mokapot score', "mokapot q-value", "mokapot PEP"])

mm_df = pd.read_csv('MokaPot_Output/metamorpheus.csv')
mm_df = mm_df.filter(['ScanNr', 'mokapot score', "mokapot q-value", "mokapot PEP"])


msf_df = pd.read_csv('MokaPot_Output/msfragger.csv')
msf_df = msf_df.filter(['ScanNr', 'mokapot score', "mokapot q-value", "mokapot PEP"])

mq_df = pd.read_csv('MokaPot_Output/maxquant.csv')
mq_df = mq_df.filter(['ScanNr', 'mokapot score', "mokapot q-value", "mokapot PEP"])

Reading in the orginial data

In [9]:
mm_original_df = get_orgininal_mm_data()
msf_original_df = get_original_msf_data()
msg_original_df = get_original_msg_data()
mq_original_df = get_original_mq_data()

../data_loader.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp_peptide'] = df.apply(lambda row: format_oxidation(row, "Modified sequence", "(Oxidation (M))"), axis=1)
../data_loader.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["temp_peptide"] = df["temp_peptide"].str[1:-1]
../data_loader.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [10]:
msf_df

,ScanNr,mokapot score,mokapot q-value,mokapot PEP
0,28519,-0.000000,0.002224,0.131193
1,28050,-0.000000,0.002224,0.131193
2,24332,-0.000000,0.002224,0.131193
3,23816,-0.062012,0.002224,0.131193
4,21914,-0.108828,0.002224,0.131193
...,...,...,...,...
13482,21467,-4.272878,0.002224,0.131193
13483,21514,-4.276380,0.002224,0.131193
13484,28339,-4.466068,0.002224,0.131193
13485,41908,-5.150720,0.002224,0.131193


In [11]:
#Switching index to ScanNr to join dataframes based on their scan numbers
msg_df = msg_df.set_index('ScanNr')
msg_original_df = msg_original_df.set_index('ScanNr')
msf_df = msf_df.set_index('ScanNr')
msf_original_df = msf_original_df.set_index('ScanNr')
mm_df = mm_df.set_index('ScanNr')
mm_original_df = mm_original_df.set_index('ScanNr')
mq_df = mq_df.set_index('ScanNr')
mq_original_df = mq_original_df.set_index('ScanNr')


In [12]:
#Joining data from the original and postMokapot into a df
MsFragger = msf_original_df.join(msf_df, how="outer")
MsFragger.to_csv("MokaPot_Output/msf_deleteMe.csv")
MsFragger

,peptide,probability,mokapot score,mokapot q-value,mokapot PEP
ScanNr,,,,,
10427,KLSAPVSQK,0.1059,-0.780285,0.002224,0.131193
10531,KSHWDVVRSIVM+15.995FGK,0.2933,-0.831055,0.002224,0.131193
10551,EDTEEHHLR,0.9912,-1.802539,0.002224,0.131193
10576,LSSAM+15.995SAAK,0.9973,-1.037314,0.002224,0.131193
10589,QVDGDNSHVEM+15.995K,0.9816,-1.344130,0.002224,0.131193
...,...,...,...,...,...
44019,GSDIIVDLEVTLEEVYAGNFVEVVR,1.0000,-1.943977,0.002224,0.131193
44027,GSDIIVDLEVTLEEVYAGNFVEVVR,0.9978,-1.737485,0.002224,0.131193
44041,VGAGAPVYM+15.995AAVLEYLTAEILELAGNAAR,1.0000,-2.216571,0.002224,0.131193


In [13]:
MsgfPlus = msg_original_df.join(msg_df, how="outer")
#MsgfPlus.to_csv("MokaPot_Output/msg_deleteMe.csv")
MsgfPlus

,peptide,QValue,mokapot score,mokapot q-value,mokapot PEP
ScanNr,,,,,
6,QQCGCQPKLCDYRCIWCQK,0.331900,-1.141648,0.302455,1.0
9,VEIPVLGR,0.433359,-1.458714,0.383631,1.0
14,DCEM+15.995YKHR,0.360605,-0.832250,0.225213,1.0
19,PYECM+15.995ECGGK,0.441487,-1.497486,0.393484,1.0
21,SDTDEEEDNK,0.454363,-1.450276,0.380983,1.0
...,...,...,...,...,...
49306,CTCDSNLEK,0.320573,-0.389669,0.116693,1.0
49317,TQSQEEELM+15.995K,0.311058,NaN,NaN,NaN
49342,LYECSECGR,0.393170,-0.457595,0.133164,1.0


In [14]:
MetaMorpheus = mm_original_df.join(mm_df, how="outer")
MetaMorpheus.to_csv("MokaPot_Output/mm_deleteMe.csv")
MetaMorpheus

,peptide,QValue,PEP,mokapot score,mokapot q-value,mokapot PEP
ScanNr,,,,,,
8872,KLSAPVSQK,0.103226,0.999999,-2.493063,0.073283,1.000000
10427,KLSAPVSQK,0.007482,0.452308,-0.110992,0.010145,0.181737
10551,EDTEEHHLR,0.011904,0.061240,0.182756,0.002995,0.051978
10576,LSSAM+15.995SAAK,0.096447,0.998882,0.115495,0.004449,0.071740
10589,QVDGDNSHVEM+15.995K,0.014952,0.000380,0.535814,0.000980,0.008995
...,...,...,...,...,...,...
44027,GSDIIVDLEVTLEEVYAGNFVEVVR,0.068295,0.910394,0.392130,0.001552,0.018165
44038,TWHKIFQESTVLHLCLREIQQQR,0.074657,0.999999,-1.907118,0.068251,1.000000
44041,VGAGAPVYM+15.995AAVLEYLTAEILELAGNAAR,0.000000,0.000041,1.647927,0.000134,0.000143


In [15]:
MaxQuant = mq_original_df.join(mq_df, how="outer")
MaxQuant.to_csv("MokaPot_Output/mq_deleteMe.csv")
MaxQuant

,peptide,PEP,mokapot score,mokapot q-value,mokapot PEP
ScanNr,,,,,
7,NEDEDSPNK,1.773420,-0.577871,0.137777,0.408407
9,VLLPAIKK,0.175843,-0.443765,0.099749,0.405990
11,SYVLALGR,1.870431,-0.293194,0.053161,0.386646
13,,NaN,1.511118,0.000225,0.000254
14,EGELMDCDGK,0.290638,-0.869219,0.199275,0.408407
...,...,...,...,...,...
49306,M+15.995ESTETEER,0.290642,-0.551482,0.131189,0.408407
49317,CMECGKAFCRVGDLR,0.131159,-1.943595,0.250650,0.408407
49331,,NaN,1.493624,0.000225,0.000279


In [25]:
megaScript = pd.concat(dict(MsFragger = MsFragger, MsgfPlus = MsgfPlus, MetaMorpheus = MetaMorpheus, MaxQuant = MaxQuant), axis=1)
megaScript

MsFragger                                                        \
         peptide probability mokapot score mokapot q-value mokapot PEP   
ScanNr                                                                   
6            NaN         NaN           NaN             NaN         NaN   
7            NaN         NaN           NaN             NaN         NaN   
9            NaN         NaN           NaN             NaN         NaN   
11           NaN         NaN           NaN             NaN         NaN   
13           NaN         NaN           NaN             NaN         NaN   
...          ...         ...           ...             ...         ...   
49331        NaN         NaN           NaN             NaN         NaN   
49335        NaN         NaN           NaN             NaN         NaN   
49342        NaN         NaN           NaN             NaN         NaN   
49347        NaN         NaN           NaN             NaN         NaN   
49354        NaN         NaN           NaN             NaN         NaN   

                   MsgfPlus                                          \
                    peptide    QValue mokapot score mokapot q-value   
ScanNr                                                                
6       QQCGCQPKLCDYRCIWCQK  0.331900     -1.141648        0.302455   
7                       NaN       NaN           NaN             NaN   
9                  VEIPVLGR  0.433359     -1.458714        0.383631   
11                      NaN       NaN           NaN             NaN   
13                      NaN       NaN           NaN             NaN   
...                     ...       ...           ...             ...   
49331                   NaN       NaN           NaN             NaN   
49335                   NaN       NaN           NaN             NaN   
49342             LYECSECGR  0.393170     -0.457595        0.133164   
49347            DEDNDEDEER  0.472461     -1.452637        0.381693   
49354              WHHEYQTR  0.148778     -0.530315        0.150900   

                    ... MetaMorpheus                                    \
       mokapot PEP  ...       QValue PEP mokapot score mokapot q-value   
ScanNr              ...                                                  
6              1.0  ...          NaN NaN           NaN             NaN   
7              NaN  ...          NaN NaN           NaN             NaN   
9              1.0  ...          NaN NaN           NaN             NaN   
11             NaN  ...          NaN NaN           NaN             NaN   
13             NaN  ...          NaN NaN           NaN             NaN   
...            ...  ...          ...  ..           ...             ...   
49331          NaN  ...          NaN NaN           NaN             NaN   
49335          NaN  ...          NaN NaN           NaN             NaN   
49342          1.0  ...          NaN NaN           NaN             NaN   
49347          1.0  ...          NaN NaN           NaN             NaN   
49354          1.0  ...          NaN NaN           NaN             NaN   

                     MaxQuant                                          \
       mokapot PEP    peptide       PEP mokapot score mokapot q-value   
ScanNr                                                                  
6              NaN        NaN       NaN           NaN             NaN   
7              NaN  NEDEDSPNK  1.773420     -0.577871        0.137777   
9              NaN   VLLPAIKK  0.175843     -0.443765        0.099749   
11             NaN   SYVLALGR  1.870431     -0.293194        0.053161   
13             NaN                  NaN      1.511118        0.000225   
...            ...        ...       ...           ...             ...   
49331          NaN                  NaN      1.493624        0.000225   
49335          NaN                  NaN      0.491633        0.000225   
49342          NaN        NaN       NaN           NaN             NaN   
49347          NaN                  NaN      1.516223        0

In [26]:
megaScript.reset_index(inplace=True)
megaScript

ScanNr MsFragger                                                        \
               peptide probability mokapot score mokapot q-value mokapot PEP   
0          6       NaN         NaN           NaN             NaN         NaN   
1          7       NaN         NaN           NaN             NaN         NaN   
2          9       NaN         NaN           NaN             NaN         NaN   
3         11       NaN         NaN           NaN             NaN         NaN   
4         13       NaN         NaN           NaN             NaN         NaN   
...      ...       ...         ...           ...             ...         ...   
35835  49331       NaN         NaN           NaN             NaN         NaN   
35836  49335       NaN         NaN           NaN             NaN         NaN   
35837  49342       NaN         NaN           NaN             NaN         NaN   
35838  49347       NaN         NaN           NaN             NaN         NaN   
35839  49354       NaN         NaN           NaN             NaN         NaN   

                  MsgfPlus                                          ...  \
                   peptide    QValue mokapot score mokapot q-value  ...   
0      QQCGCQPKLCDYRCIWCQK  0.331900     -1.141648        0.302455  ...   
1                      NaN       NaN           NaN             NaN  ...   
2                 VEIPVLGR  0.433359     -1.458714        0.383631  ...   
3                      NaN       NaN           NaN             NaN  ...   
4                      NaN       NaN           NaN             NaN  ...   
...                    ...       ...           ...             ...  ...   
35835                  NaN       NaN           NaN             NaN  ...   
35836                  NaN       NaN           NaN             NaN  ...   
35837            LYECSECGR  0.393170     -0.457595        0.133164  ...   
35838           DEDNDEDEER  0.472461     -1.452637        0.381693  ...   
35839             WHHEYQTR  0.148778     -0.530315        0.150900  ...   

      MetaMorpheus                                                 MaxQuant  \
            QValue PEP mokapot score mokapot q-value mokapot PEP    peptide   
0              NaN NaN           NaN             NaN         NaN        NaN   
1              NaN NaN           NaN             NaN         NaN  NEDEDSPNK   
2              NaN NaN           NaN             NaN         NaN   VLLPAIKK   
3              NaN NaN           NaN             NaN         NaN   SYVLALGR   
4              NaN NaN           NaN             NaN         NaN              
...            ...  ..           ...             ...         ...        ...   
35835          NaN NaN           NaN             NaN         NaN              
35836          NaN NaN           NaN             NaN         NaN              
35837          NaN NaN           NaN             NaN         NaN        NaN   
35838          NaN NaN           NaN             NaN         NaN              
35839          NaN NaN           NaN             NaN         NaN        NaN   

                                                           
            PEP mokapot score mokapot q-value mokapot PEP  
0           NaN           NaN             NaN         NaN  
1      1.773420     -0.577871        0.137777    0.408407  
2      0.175843     -0.443765        0.099749    0.405990  
3      1.870431     -0.293194        0.053161    0.386646  
4           NaN      1.511118        0.000225    0.000254  
...         ...           ...             ...         ...  
35835       NaN      1.493624        0.000225    0.000279  
35836       NaN      0.491633        0.000225    0.048132  
35837       NaN           NaN             NaN         NaN  
35838       NaN      1.516223        0.000225    0.000248  
35839       NaN           NaN             NaN         NaN  

[35840 rows x 22 columns]

In [27]:
megaScript.to_csv("MegaScript.csv")

In [19]:
megaScript.columns.get_level_values(0)

Index(['ScanNr', 'MsFragger', 'MsFragger', 'MsFragger', 'MsFragger',
       'MsFragger', 'MsgfPlus', 'MsgfPlus', 'MsgfPlus', 'MsgfPlus', 'MsgfPlus',
       'MetaMorpheus', 'MetaMorpheus', 'MetaMorpheus', 'MetaMorpheus',
       'MetaMorpheus', 'MetaMorpheus', 'MaxQuant', 'MaxQuant', 'MaxQuant',
       'MaxQuant', 'MaxQuant'],
      dtype='object')

In [20]:
#Why is there a difference in length in msgplus? We already know that mm was a little different. 

print("msf_orig: " + str(len(msf_original_df)))
print("msf_post: " + str(len(msf_df)))

print("msg_orig: " + str(len(msg_original_df)))
print("msg_post: " + str(len(msg_df)))

print("mq_orig: " + str(len(mq_original_df)))
print("mq_post: " + str(len(mq_df)))

print("mm_orig: " + str(len(mm_original_df)))
print("mm_post: " + str(len(mm_df)))

msf_orig: 13487
msf_post: 13487
msg_orig: 23665
msg_post: 23038
mq_orig: 31604
mq_post: 31604
mm_orig: 16306
mm_post: 16140
